In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import pingouin as pg
from scipy.stats import shapiro, normaltest
import numpy as np
import kdephys as kde

import warnings
warnings.filterwarnings('ignore')

probe_ord = ['NNXr', 'NNXo']

from acr.plots import lrg
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from acr.utils import *

In [ ]:
%connect_info

In [ ]:
scd = {}
scd['ACR_18'] = [3, 4, 5, 10]


In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'acr'
MAIN_COLOR = SOM_BLUE if SUBJECT_TYPE == 'som' else ACR_BLUE

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
subjects, exps = pu.remove_subject('ACR_16', subjects, exps)
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/firing_rates'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False, extra_rebounds=False)
reb_hd = {}
for subject, exp in zip(subjects, exps):
    reb_hd[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=True)

In [ ]:
fr_dfs = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp)
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject])
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, reb_hd[subject], label_col='reb_condition')
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='full_bl', ref_to_val='True', col_to_relativize='fr', on=['probe', 'channel'])
    fr_dfs[subject] = fr
frdf_all = pl.concat(fr_dfs.values())

In [ ]:
fr_dfs_prb = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject])
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, reb_hd[subject], label_col='reb_condition')
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='condition', ref_to_val='early_sd', col_to_relativize='fr', on=['probe'])
    fr_dfs_prb[subject] = fr

# STIMULATION FIRING RATE

In [ ]:
frdf_all = pl.concat(fr_dfs.values())
frdf_prb = pl.concat(fr_dfs_prb.values())

In [ ]:
scd = {}
frdf = pu.drop_sub_channels(frdf_all, scd)
#frdf_prb = pu.drop_sub_channels(frdf_all_prb, scd)

In [ ]:
stimfr = frdf.filter(pl.col('condition')=='stim')
#stimfr = pu.drop_sub_channels(stimfr, scd)
stimfr_means = stimfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
stimfr_means = stimfr_means.sort(['subject', 'probe'])
stim_chan_means = stimfr.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
g = sns.catplot(data=stim_chan_means, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_during_entire_stim'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'


stimfr = frdf_prb.filter(pl.col('condition')=='stim')
stimfr_means = stimfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
stimfr_means = stimfr_means.sort(['subject', 'probe'])

nnxr = stimfr_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = stimfr_means.prb('NNXo')['fr_rel'].to_numpy()



f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# REBOUND FIRING RATE

In [ ]:
fr_dfs_prb = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject])
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, reb_hd[subject], label_col='reb_condition')
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='full_bl', ref_to_val='True', col_to_relativize='fr', on=['probe'])
    fr_dfs_prb[subject] = fr
frdf_prb = pl.concat(fr_dfs_prb.values())

In [ ]:
rebfr = frdf.filter(pl.col('condition')=='rebound')
rebfr_means = rebfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr_means = rebfr_means.sort(['subject', 'probe'])
reb_chan_means = rebfr.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
g = sns.catplot(data=reb_chan_means, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_during_NREM_REBOUND_1HR'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

rebfr = frdf_prb.filter(pl.col('condition')=='rebound')
rebfr_means = rebfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr_means = rebfr_means.sort(['subject', 'probe'])

nnxr = rebfr_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# Recovery

In [ ]:
rebfr2 = frdf_prb.filter(pl.col('reb_condition')=='reb2')
rebfr2_means = rebfr2.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr2_means = rebfr2_means.sort(['subject', 'probe'])
#reb_chan_means2 = rebfr2.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=reb_chan_means, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_after_NREM_REBOUND_HOUR-2'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rebfr2_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr2_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rebfr3 = frdf_prb.filter(pl.col('reb_condition')=='reb3')
rebfr3_means = rebfr3.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr3_means = rebfr3_means.sort(['subject', 'probe'])
#reb_chan_means3 = rebfr3.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=reb_chan_means3, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_after_NREM_REBOUND_HOUR-3'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rebfr3_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr3_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rebfr4 = frdf_prb.filter(pl.col('reb_condition')=='reb4')
rebfr4_means = rebfr4.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr4_means = rebfr4_means.sort(['subject', 'probe'])
#reb_chan_means4 = rebfr4.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=reb_chan_means4, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_after_NREM_REBOUND_HOUR-4'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rebfr4_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr4_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
rebfr5 = frdf_prb.filter(pl.col('reb_condition')=='reb5')
rebfr5_means = rebfr5.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr5_means = rebfr5_means.sort(['subject', 'probe'])
#reb_chan_means5 = rebfr5.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=reb_chan_means5, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_after_NREM_REBOUND_HOUR-5'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rebfr5_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr5_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

## OVER Time

In [ ]:
frdf_prb = frdf_prb.sort(['datetime', 'subject', 'probe'])
frsubs = frdf_prb.group_by_dynamic(index_column='datetime', every='20s', closed='left', group_by=['subject', 'probe'], start_by='datapoint').agg(pl.col('fr_rel').mean())

In [ ]:
dfs = []
for subject in frsubs['subject'].unique():
    df = frsubs.filter(pl.col('subject') == subject)
    df = acr.hypnogram_utils.label_df_with_states(df, full_hyps[subject])
    dfs.append(df)
frsubs = pl.concat(dfs)
frsubs = frsubs.sort(['subject', 'probe', 'datetime'])

In [ ]:
dfs = []
for subject in frsubs['subject'].unique():
    rs = hyp_dicts[subject]['rebound']['start_time'].min()
    df = frsubs.filter(pl.col('subject') == subject).ts(rs, rs+pd.Timedelta(hours=24)).st('NREM').sort('datetime')
    for probe in ['NNXo', 'NNXr']:
        pdf = df.prb(probe)
        bouts = np.arange(0, len(pdf), 1)
        bouts = bouts/3
        pdf = pdf.with_columns(bout=pl.lit(bouts))
        print(subject, len(bouts))
        dfs.append(pdf)
frreb = pl.concat(dfs)
frreb_bouts = frreb.group_by(['subject', 'bout', 'probe']).agg(pl.col('fr_rel').mean()).sort(['subject', 'probe', 'bout'])

In [ ]:
fig_id = 'rebound_FIRING-RATE_recovery_over_time'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(16, 8))
sns.lineplot(data=frreb_bouts.to_pandas(), x='bout', y='fr_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, MAIN_COLOR], ax=ax, errorbar='se')
ax.axhline(1.0, color='red', linestyle='--')
ax.set_xlim(0, 300)
ax.set_ylabel('')
ax.set_xlabel('Time (min)')
ax.legend().remove()
ax.set_ylim(0.62, 1.4)
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
for subject in subjects:
    df = frreb_bouts.filter(pl.col('subject') == subject)
    f, ax = plt.subplots(1, 1, figsize=(28, 8))
    sns.lineplot(data=df.to_pandas(), x='bout', y='fr_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, MAIN_COLOR], ax=ax)
    ax.set_title(f'{subject}')
    plt.show()

# REM

In [ ]:
rdfs = []
for subject in subjects:
    subdf = frdf_prb.filter(pl.col('subject') == subject)
    reb_start = hyp_dicts[subject]['rebound']['start_time'].min()
    reb_end = hyp_dicts[subject]['rebound']['end_time'].max()
    subdf = subdf.ts(reb_start, reb_end).filter(pl.col('state') == 'REM')
    rdfs.append(subdf)
rdf = pl.concat(rdfs)

In [ ]:
rdf_means = rdf.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())

In [ ]:
fig_id = 'firing_rate_during_REM_REBOUND_1HR'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rdf_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rdf_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = False

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# BY DEPTH - NOT READY FOR PRODUCTION

In [ ]:
# NOTE: THIS ANALYSIS IS NOT COMPLETED, NOT INCLUDED IN ANY FINAL VERSION.

In [ ]:
nbroot = f'{acr.utils.PAPER_FIGURE_ROOT}/firing_rates/by_depth'

In [ ]:
rebfr = frdf.filter(pl.col('condition')=='rebound')
rebfr = acr.info_pipeline.label_df_sub_infra(rebfr)
reb_fr_means = rebfr.group_by(['subject', 'probe', 'label']).agg(pl.col('fr_rel').mean()).sort(['subject', 'probe', 'label'])

In [ ]:
fig_id = 'firing_rate_1hr_NREM_rebound--infragranular'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxr = reb_fr_means.filter(pl.col('label')=='infragranular').prb('NNXr')['fr_rel'].to_numpy()
nnxo = reb_fr_means.filter(pl.col('label')=='infragranular').prb('NNXo')['fr_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'firing_rate_1hr_NREM_rebound--granular'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxr = reb_fr_means.filter(pl.col('label')=='granular').prb('NNXr')['fr_rel'].to_numpy()
nnxo = reb_fr_means.filter(pl.col('label')=='granular').prb('NNXo')['fr_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'firing_rate_1hr_NREM_rebound--subgranular'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxr = reb_fr_means.filter(pl.col('label')=='subgranular').prb('NNXr')['fr_rel'].to_numpy()
nnxo = reb_fr_means.filter(pl.col('label')=='subgranular').prb('NNXo')['fr_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# ON-PERIOD-Specific

In [ ]:
oodfs = {}
for subject, exp in zip(subjects, exps):
    print(subject)
    hdf = dag.compute_hybrid_off_df(subject, exp, chan_threshold=12)
    hdf = acr.oo_utils.enhance_oodf(hdf, full_hyps[subject], hyp_dicts[subject])
    oodfs[subject] = hdf

In [ ]:
on_dfs = {}
for subject in oodfs.keys():
    on_df = dag.compute_hybrid_on_df(oodfs[subject])
    on_df = acr.oo_utils.enhance_oodf(on_df, full_hyps[subject], hyp_dicts[subject])
    on_dfs[subject] = on_df
ondf = pl.concat(on_dfs.values())

In [ ]:
spk = {}
for subject in on_dfs.keys():
    bin_counts = dag.load_binned_spikes(subject, acr.utils.sub_swi_exps[subject][0])
    spk[subject] = bin_counts

In [ ]:
better_on_dfs = []
for subject in on_dfs.keys():
    for probe in ['NNXr', 'NNXo']:
        on = ondf.filter(pl.col('subject')==subject).prb(probe)
        spk_counts = spk[subject][probe]
        on = add_global_spk_rate(on, spk_counts)
        on = acr.oo_utils.relativize_oodf(on, ref_to_col='full_bl', ref_to_val='True', col_to_relativize='spk_rate')
        better_on_dfs.append(on)
onn = pl.concat(better_on_dfs)

In [ ]:
onn_cond = onn.filter(pl.col('condition') != 'None')
onn_means = onn_cond.group_by(['subject', 'probe', 'condition']).agg(pl.col('spk_rate_rel').mean())
reb_on_fr = onn_cond.filter(pl.col('condition')=='rebound')
rebon_means = reb_on_fr.group_by(['subject', 'probe']).agg(pl.col('spk_rate_rel').mean()).sort(['subject', 'probe'])

In [ ]:
total_on_time = onn_cond.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration').sum()).sort(['subject', 'probe', 'condition'])

In [ ]:
f, ax = plt.subplots(figsize=(25, 10))
sns.stripplot(data=onn_means, x='condition', y='spk_rate_rel', hue='probe', order=['early_bl', 'circ_bl', 'rebound'])

In [ ]:
nnxr = rebon_means.prb('NNXr')['spk_rate_rel'].to_numpy()
nnxo = rebon_means.prb('NNXo')['spk_rate_rel'].to_numpy()
plt.rcdefaults()
plt.style.use('fivethirtyeight')
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)

In [ ]:
t50_on_dfs = []
for subject in on_dfs.keys():
    for probe in ['NNXr', 'NNXo']:
        on = ondf.filter(pl.col('subject')==subject).prb(probe)
        spk_counts = spk[subject][probe]
        on = add_global_spk_rate(on, spk_counts, use_ends=0.05)
        on = acr.oo_utils.relativize_oodf(on, ref_to_col='full_bl', ref_to_val='True', col_to_relativize='spk_rate')
        better_on_dfs.append(on)
onn_50 = pl.concat(better_on_dfs)